# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
import matplotlib.mlab as mlab

%matplotlib inline

fs = 125
minfreq = 40/60.0
maxfreq = 240/60.0

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):   
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)

    # Loading the reference file
    ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)
    
    # Bandpass ppg and acc signals 
    ppg = bandpass_filter(ppg)
    accx = bandpass_filter(accx)
    accy = bandpass_filter(accy)
    accz = bandpass_filter(accz)
        
    # Calculate Power Spectral Density estimates for each signal
    psd_ppg, freqs_ppg = cal_spectogram(signal=ppg)
    psd_accx, freqs_accx = cal_spectogram(signal=accx)
    psd_accy, freqs_accy = cal_spectogram(signal=accy)
    psd_accz, freqs_accz = cal_spectogram(signal=accz)
    
    # Get frequencies and time steps
    freqs = freqs_ppg.shape[0]
    time_steps = psd_ppg.shape[1]
    
    # Sorted index value for signals 
    ppg_index = (-psd_ppg).argsort(axis=0)
    accx_index = (-psd_accx).argsort(axis=0)
    accy_index = (-psd_accy).argsort(axis=0)
    accz_index = (-psd_accz).argsort(axis=0)

    estimated_freq = []
    for t in range(time_steps):
        for freq in range(freqs):
            i=0
            if freq == 2:
                estimated_freq.append(freqs_ppg[ppg_index[freq][t]])
                break
            elif np.all([(freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accx_index[i][t]]), 
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accy_index[i][t]]), 
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accz_index[i][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accx_index[i+1][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accy_index[i+1][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accz_index[i+1][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accx_index[i+2][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accy_index[i+2][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accz_index[i+2][t]])]):
                estimated_freq.append(freqs_ppg[ppg_index[freq][t]])
                break
            
    # Claculate confidence
    confidence = []
    for i in range(len(estimated_freq)):
        snr = calc_snr(ppg, estimated_freq[i])
        confidence.append(snr)
    
    pre = np.array(estimated_freq) * 60
    
    # Claculate the absolute error 
    error_array = np.abs(ground_truth - pre)
    conf_array = np.array(confidence)

    # Return per-estimate absolute error and confidence as a 2-tuple of numpy arrays.
    return error_array, conf_array


def bandpass_filter(signal):
    pass_band=(minfreq, maxfreq)
    b, a = scipy.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

def calc_snr(signal, hr_f):
    n = len(signal)*2
    harmonic_f = hr_f * 2
    fft_mag = np.abs(np.fft.rfft(signal, n))
    freqs = np.fft.rfftfreq(n, 1/fs)
    window_f = 5/60
    fundamental_frequency_window = (freqs > hr_f - window_f) & (freqs < hr_f + window_f)
    harmonic_frequency_window = (freqs > harmonic_f - window_f) & (freqs < harmonic_f + window_f)
    signal = np.sum(fft_mag[(fundamental_frequency_window) | (harmonic_frequency_window)])
    noise = np.sum(fft_mag[~ ((fundamental_frequency_window) | (harmonic_frequency_window))])
    snr = signal / noise
    
    return snr
    
    
def cal_spectogram(signal):
    psd, freqs, t = mlab.specgram(signal, NFFT=8*fs, Fs=fs, noverlap=6*fs, pad_to=12*fs)
    psd = psd[(freqs >= minfreq) & (freqs <= maxfreq)]
    freqs = freqs[(freqs >= minfreq) & (freqs <= maxfreq)]
    return psd, freqs